In [1]:
#!/env/bin/python

import init

import findspark
findspark.init()

import os
import pandas as pd
import numpy as np
from pyspark.sql import *
from pyspark import *
import pyspark.sql.functions as f
from timeit import default_timer as timer
from RecsysSchema import RecsysSchema
from pyrecdp.data_processor import *
from pyrecdp.utils import *

path_prefix = "hdfs://"
categorified_folder = "/recsys2021/data_all_with_text"
dicts_folder = "/recsys2021/recsys_dicts"
recsysSchema = RecsysSchema()

##### 1. Start spark and initialize data processor #####
t0 = timer()
spark = SparkSession.builder.master('yarn')\
    .appName("Recsys2021_data_viewer")\
    .getOrCreate()

schema = recsysSchema.toStructType()

categorified_df = spark.read.parquet(path_prefix + categorified_folder)
# load pre-generated dicts and combined with current data
dict_names = ['hashtags', 'language', 'present_domains', 'present_links', 'tweet_id', 'user_id']
dict_dfs = dict((name, spark.read.parquet("%s/%s/%s" % (path_prefix, dicts_folder, name))) for name in dict_names)


/mnt/nvme2/chendi/BlueWhale/frameworks.bigdata.bluewhale/examples/RecSys/submission/RecDP


In [ ]:
categorified_df.show(1, vertical=True, truncate=False)

In [9]:
for col_name, dict_df in dict_dfs.items():
    print("%s has numItems %d, schema is dict_col, dict_col_id" % (col_name, dict_df.count()))
    # dict_df.show(1)

hashtags has numItems 6653969
+--------------------+-----------+
|            dict_col|dict_col_id|
+--------------------+-----------+
|A338C5C1637BEDA84...|          0|
+--------------------+-----------+
only showing top 1 row

language has numItems 66
+--------------------+-----------+
|            dict_col|dict_col_id|
+--------------------+-----------+
|488B32D24BD4BB441...|          0|
+--------------------+-----------+
only showing top 1 row

present_domains has numItems 900139
+--------------------+-----------+
|            dict_col|dict_col_id|
+--------------------+-----------+
|F595B7DE8992A3D8C...|          0|
+--------------------+-----------+
only showing top 1 row

present_links has numItems 15025550
+--------------------+-----------+
|            dict_col|dict_col_id|
+--------------------+-----------+
|DAEA7BD8C379EA9E9...|          0|
+--------------------+-----------+
only showing top 1 row

tweet_id has numItems 323033151
+--------------------+-----------+
|         

In [2]:
add_original_df = categorified_df.join(\
    dict_dfs['user_id'].hint('shuffle_hash'), \
    categorified_df.engaged_with_user_id == dict_dfs['user_id'].dict_col_id, \
    'left').withColumnRenamed('dict_col', 'original_engaged_with_user_id').drop('dict_col_id')
original_user_id_joined_folder = "/recsys2021/processed_with_text_and_userid_tmp_0"
add_original_df.write.format('parquet').mode('overwrite').save(path_prefix + original_user_id_joined_folder)
add_original_df = spark.read.parquet(path_prefix + original_user_id_joined_folder)

In [4]:
add_original_df = add_original_df.join(\
    dict_dfs['user_id'].hint('shuffle_hash'), \
    add_original_df.enaging_user_id == dict_dfs['user_id'].dict_col_id, \
    'left').withColumnRenamed('dict_col', 'original_enaging_user_id').drop('dict_col_id')
original_user_id_joined_folder = "/recsys2021/processed_with_text_and_userid"
add_original_df.write.format('parquet').mode('overwrite').save(path_prefix + original_user_id_joined_folder)

In [5]:
original_user_id_joined_folder = "/recsys2021/processed_with_text_and_userid"
add_original_df = spark.read.parquet(path_prefix + original_user_id_joined_folder)
add_original_df.show(1, vertical=True, truncate=False)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 tweet_id                           | 317961                                                                                                                                                                                                                                                                                                                                                                                              
 text_tokens                        | 101	100	196	107	146	27874	10271	112	187	10105	31434	20412	118	30265	17038	43729	164	119	119	119	166	146	2787